In [48]:
import pandas as pd
import geopandas

In [71]:
import json

csv = pd.read_csv('csvallmetro.csv')
atlantageocodes = pd.DataFrame(csv['GEOID'].drop_duplicates())

# display(atlantageocodes)

In [72]:
# =SUM(SUMIFS(E:E,B:B,H2,D:D,J2,C:C,{"Less than $10,000","$10,000 to $19,999","$20,000 to $34,999"}))
# =SUM(SUMIFS(E:E,B:B,H9,D:D,J9,C:C,{"$35,000 to $49,999","$50,000 to $74,999"}))
# =SUM(SUMIFS(E:E,B:B,H16,D:D,J16,C:C,{"$75,000 to $99,999","$100,000 or more"}))

# =SUM(SUMIFS(D:D,A:A,G2,B:B,H2,C:C,{"Less than 20.0 percent","20.0 to 24.9 percent","25.0 to 29.9 percent"}))
# =SUM(SUMIFS(D:D,A:A,G3,B:B,H3,C:C,{"30.0 to 34.9 percent","35.0 to 39.9 percent","40.0 to 49.9 percent"}))
# =SUM(SUMIFS(D:D,A:A,G4,B:B,H4,C:C,{"50.0 percent or more"}))

income_mapping = {
        "Less than $10,000":'Low Income',
        "$10,000 to $19,999":'Low Income',
        "$20,000 to $34,999":'Low Income',
            "$35,000 to $49,999":'Med Income',
            "$50,000 to $74,999":'Med Income',
                "$75,000 to $99,999":'High Income',
                "$100,000 or more":'High Income',
    }

rent_mapping = {
        "Less than 20.0 percent":'Low Burden',
        "20.0 to 24.9 percent":'Low Burden',
        "25.0 to 29.9 percent":'Low Burden',
            "30.0 to 34.9 percent":'Med Burden',
            "35.0 to 39.9 percent":'Med Burden',
            "40.0 to 49.9 percent":'Med Burden',
                "50.0 percent or more":'High Burden',
    }

csv['IncomeG'] = csv['Income'].replace(income_mapping)
csv['RentG'] = csv['Rent as % of Income'].replace(rent_mapping)

In [73]:
# display(csv)

df = csv.groupby(['GEOID','YEAR','IncomeG','RentG']).sum()
df.to_csv('temp.csv')
df = pd.read_csv('temp.csv')

In [74]:
display(df.head())
type(df)




GEOID  YEAR      IncomeG        RentG  HHs
0  13013180103  2014  High Income  High Burden    0
1  13013180103  2014  High Income   Low Burden   38
2  13013180103  2014  High Income   Med Burden    0
3  13013180103  2014   Low Income  High Burden    0
4  13013180103  2014   Low Income   Low Burden    0

pandas.core.frame.DataFrame

In [115]:
totalHHs = csv.groupby(['GEOID','YEAR']).sum()

display(totalHHs.head())

totalHHs.to_csv('temp.csv')
totalHHs = pd.read_csv('temp.csv')

display(totalHHs.head())

totalHHs = totalHHs[totalHHs['YEAR']==2017]



HHs
GEOID       YEAR     
13013180103 2014  109
            2015  157
            2016  162
            2017  219
13013180104 2014  116

GEOID  YEAR  HHs
0  13013180103  2014  109
1  13013180103  2015  157
2  13013180103  2016  162
3  13013180103  2017  219
4  13013180104  2014  116

In [116]:
#Calculate % rent burdened all incomes
totalHHspercent = csv.groupby(['GEOID','YEAR','RentG']).sum()

display(totalHHs.head())

totalHHspercent.to_csv('temp.csv')
totalHHspercent = pd.read_csv('temp.csv')

display(totalHHs.head())

totalHHspercent.index = totalHHspercent['GEOID']
totalHHspercent.rename(columns={"GEOID": "GEOID2"}, inplace=True)

totalHHspercent = totalHHspercent[(totalHHspercent['YEAR']==2017) & (totalHHspercent['RentG']=='High Burden')]['HHs'] / totalHHspercent[(totalHHspercent['YEAR']==2017)].groupby(['GEOID']).sum()['HHs']

# display(totalHHs[(totalHHs['YEAR']==2017) & (totalHHs['RentG']=='High Burden')]['HHs'])

# display(totalHHs[(totalHHs['YEAR']==2017)].groupby(['GEOID']).sum()['HHs'])

totalHHspercent = pd.DataFrame(totalHHspercent)
totalHHspercent['GEOID'] = totalHHspercent.index
totalHHspercent.rename(columns={"HHs": "HHPercentExtreme"}, inplace=True)

display(totalHHspercent)

GEOID  YEAR  HHs
3   13013180103  2017  219
7   13013180104  2017  161
11  13013180105  2017   82
15  13013180106  2017  287
19  13013180107  2017  400

GEOID  YEAR  HHs
3   13013180103  2017  219
7   13013180104  2017  161
11  13013180105  2017   82
15  13013180106  2017  287
19  13013180107  2017  400

HHPercentExtreme        GEOID
GEOID                                     
13013180103          0.150685  13013180103
13013180104          0.273292  13013180104
13013180105          0.207317  13013180105
13013180106          0.226481  13013180106
13013180107          0.192500  13013180107
13013180108          0.135417  13013180108
13013180203          0.283761  13013180203
13013180204          0.230599  13013180204
13013180205          0.308005  13013180205
13013180206          0.172170  13013180206
13013180301          0.094203  13013180301
13013180302          0.044000  13013180302
13013180303          0.233974  13013180303
13013180401          0.289474  13013180401
13013180402          0.151844  13013180402
13013180501          0.170667  13013180501
13013180502          0.196078  13013180502
13013180503          0.035971  13013180503
13015960101          0.088710  13015960101
13015960102          0.291892  13015960102
13015960200          0.102583  13015960200
13015960300          0.256065  13015960300
13015960401          0.150746  13015960401
13015960402          0.172348  13015960402
13015960500          0.318555  13015960500
13015960600          0.200791  13015960600
13015960700          0.198886  13015960700
13015960801          0.426735  13015960801
13015960802          0.029801  13015960802
13015960803          0.312261  13015960803
...                       ...          ...
13247060405          0.027778  13247060405
13247060406          0.203320  13247060406
13247060407          0.242604  13247060407
13255160100          0.159490  13255160100
13255160200          0.174216  13255160200
13255160300          0.654054  13255160300
13255160400          0.250295  13255160400
13255160500          0.203863  13255160500
13255160600          0.196347  13255160600
13255160700          0.229992  13255160700
13255160800          0.314691  13255160800
13255160900          0.234295  13255160900
13255161000          0.393519  13255161000
13255161100          0.296970  13255161100
13255161200          0.293850  13255161200
13297110100          0.185022  13297110100
13297110200          0.391447  13297110200
13297110300          0.274749  13297110300
13297110400          0.408704  13297110400
13297110503          0.187500  13297110503
13297110504          0.186722  13297110504
13297110505          0.142857  13297110505
13297110506          0.084507  13297110506
13297110507          0.124016  13297110507
13297110508          0.125561  13297110508
13297110601          0.079208  13297110601
13297110602          0.232493  13297110602
13297110603          0.232446  13297110603
13297110700          0.315391  13297110700
13297110800          0.351402  13297110800

[947 rows x 2 columns]

In [128]:
with open('ga.geojson', 'r') as f:
    data = json.load(f)


# for idx, row in df.iterrows():
# #     display(row)
#     GEOID = str(row['GEOID'])
#     YEAR = row['YEAR']
#     for feat in data['features']:
#         feat1 = feat['properties']['GEOID']
#         # print(row['GEOID'])
#         #print(row.to_dict())
#         if feat1 == GEOID:
#             feat['properties'][YEAR] = row.to_dict()
# #             print('added' + GEOID)
            
for idx, row in totalHHs.iterrows():
#     display(row)
    GEOID = str(row['GEOID'])
#     YEAR = row['YEAR']
    for feat in data['features']:
        feat1 = feat['properties']['GEOID']
        # print(row['GEOID'])
        #print(row.to_dict())
        if feat1 == GEOID:
            feat['properties']['HHs'] = str(row['HHs'])
#             print('added' + GEOID)

for idx, row in totalHHspercent.iterrows():
#     display(row)
    GEOID = str(int(row['GEOID']))
    print(GEOID)
    #YEAR = row['YEAR']
    for feat in data['features']:
        feat1 = feat['properties']['GEOID']
#         print(GEOID)
        #print(row.to_dict())
        if feat1 == GEOID:
            feat['properties']['HHPercentExtreme'] = str(row['HHPercentExtreme'])
            print('added' + GEOID)

#Write result to a new file
with open('new2.geojson', 'w') as f:
    json.dump(data, f)


13013180103
added13013180103
13013180104
added13013180104
13013180105
added13013180105
13013180106
added13013180106
13013180107
added13013180107
13013180108
added13013180108
13013180203
added13013180203
13013180204
added13013180204
13013180205
added13013180205
13013180206
added13013180206
13013180301
added13013180301
13013180302
added13013180302
13013180303
added13013180303
13013180401
added13013180401
13013180402
added13013180402
13013180501
added13013180501
13013180502
added13013180502
13013180503
added13013180503
13015960101
added13015960101
13015960102
added13015960102
13015960200
added13015960200
13015960300
added13015960300
13015960401
added13015960401
13015960402
added13015960402
13015960500
added13015960500
13015960600
added13015960600
13015960700
added13015960700
13015960801
added13015960801
13015960802
added13015960802
13015960803
added13015960803
13015960901
added13015960901
13015960902
added13015960902
13015961000
added13015961000
13045910101
added13045910101
13045910103
ad

13113140405
added13113140405
13113140406
added13113140406
13113140407
added13113140407
13113140408
added13113140408
13113140501
added13113140501
13113140502
added13113140502
13117130101
added13117130101
13117130102
added13117130102
13117130103
added13117130103
13117130104
added13117130104
13117130105
added13117130105
13117130201
added13117130201
13117130202
added13117130202
13117130203
added13117130203
13117130204
added13117130204
13117130205
added13117130205
13117130301
added13117130301
13117130302
added13117130302
13117130303
added13117130303
13117130304
added13117130304
13117130305
added13117130305
13117130306
added13117130306
13117130307
added13117130307
13117130403
added13117130403
13117130404
added13117130404
13117130405
added13117130405
13117130406
added13117130406
13117130408
added13117130408
13117130409
added13117130409
13117130410
added13117130410
13117130503
added13117130503
13117130504
added13117130504
13117130505
added13117130505
13117130506
added13117130506
13117130507
ad

added13135050520
13135050521
added13135050521
13135050522
added13135050522
13135050523
added13135050523
13135050524
added13135050524
13135050525
added13135050525
13135050526
added13135050526
13135050527
added13135050527
13135050528
added13135050528
13135050529
added13135050529
13135050530
added13135050530
13135050531
added13135050531
13135050532
added13135050532
13135050533
added13135050533
13135050534
added13135050534
13135050535
added13135050535
13135050536
added13135050536
13135050537
added13135050537
13135050538
added13135050538
13135050539
added13135050539
13135050540
added13135050540
13135050541
added13135050541
13135050542
added13135050542
13135050543
added13135050543
13135050544
added13135050544
13135050545
added13135050545
13135050546
added13135050546
13135050547
added13135050547
13135050548
added13135050548
13135050549
added13135050549
13135050605
added13135050605
13135050606
added13135050606
13135050607
added13135050607
13135050608
added13135050608
13135050609
added131350506

In [11]:
df.to_csv('final_summary.csv')

In [129]:
geotable = geopandas.read_file('new2.geojson')

display(geotable.head())
#geotable = geotable.astype('int64').dtypes
geotable[['GEOID']] = geotable[['GEOID']].astype('int64')

geotable2 = geotable.merge(atlantageocodes, on='GEOID')

# gdf = geopandas.GeoDataFrame(geotable2, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

display(geotable2.head())

# geotable2 = geotable2[~geotable2.geometry.str.contains("MULTIPOLYGON")]


STATEFP COUNTYFP TRACTCE        GEOID    NAME             NAMELSAD  MTFCC  \
0      13      153  020105  13153020105  201.05  Census Tract 201.05  G5020   
1      13      153  020108  13153020108  201.08  Census Tract 201.08  G5020   
2      13      089  023315  13089023315  233.15  Census Tract 233.15  G5020   
3      13      089  023421  13089023421  234.21  Census Tract 234.21  G5020   
4      13      089  021504  13089021504  215.04  Census Tract 215.04  G5020   

  FUNCSTAT       ALAND    AWATER     INTPTLAT      INTPTLON   HHs  \
0        S   5620037.0   89197.0  +32.6412869  -083.6449301  None   
1        S  10732046.0   65358.0  +32.6404762  -083.6768373  None   
2        S  23296834.0  363011.0  +33.7542712  -084.0659786   235   
3        S   6124825.0  136878.0  +33.6850994  -084.2587867  1723   
4        S   3732018.0   39771.0  +33.8087264  -084.3224789  1621   

      HHPercentExtreme                                           geometry  
0                 None  POLYGON ((-83.660753 32.635764, -83.660735 32....  
1                 None  POLYGON ((-83.69864699999999 32.659458, -83.69...  
2  0.37872340425531914  POLYGON ((-84.10821900000001 33.758834, -84.10...  
3   0.2199651770168311  POLYGON ((-84.277354 33.674202, -84.277171 33....  
4  0.30351634793337445  POLYGON ((-84.338874 33.799947, -84.338553 33....

STATEFP COUNTYFP TRACTCE        GEOID    NAME             NAMELSAD  MTFCC  \
0      13      089  023315  13089023315  233.15  Census Tract 233.15  G5020   
1      13      089  023421  13089023421  234.21  Census Tract 234.21  G5020   
2      13      089  021504  13089021504  215.04  Census Tract 215.04  G5020   
3      13      089  023419  13089023419  234.19  Census Tract 234.19  G5020   
4      13      089  021411  13089021411  214.11  Census Tract 214.11  G5020   

  FUNCSTAT       ALAND    AWATER     INTPTLAT      INTPTLON   HHs  \
0        S  23296834.0  363011.0  +33.7542712  -084.0659786   235   
1        S   6124825.0  136878.0  +33.6850994  -084.2587867  1723   
2        S   3732018.0   39771.0  +33.8087264  -084.3224789  1621   
3        S  13219799.0  241092.0  +33.6493730  -084.2278033   727   
4        S   1667281.0       0.0  +33.8667772  -084.3293120  1070   

      HHPercentExtreme                                           geometry  
0  0.37872340425531914  POLYGON ((-84.10821900000001 33.758834, -84.10...  
1   0.2199651770168311  POLYGON ((-84.277354 33.674202, -84.277171 33....  
2  0.30351634793337445  POLYGON ((-84.338874 33.799947, -84.338553 33....  
3  0.10041265474552957  POLYGON ((-84.25414499999999 33.64708, -84.254...  
4  0.15327102803738318  POLYGON ((-84.338921 33.860744, -84.3379610000...

In [130]:
type(geotable2)
# geotable2.to_csv('testgeo.csv')
# geotable2 = geopandas.read_file('testgeo.csv')

geotable2.to_file("metroatlanta.geojson", driver='GeoJSON')